In [1]:
pip install openpyxl

In [47]:
import os
import pandas as pd
from tqdm import tqdm

def get_all_files(directory, extension):
    matching_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(extension.lower()):
                matching_files.append(os.path.join(root, file))
    return matching_files

def process_data(gps_dir, ambulance_dir, output_file='wynik.csv'):
    # Ładowanie plików .xls/.xlsx z karetkami
    xls_files = get_all_files(ambulance_dir, '.xlsx')
    if not xls_files:
        raise ValueError("Brak plików .xlsx z wyjazdami!")

    print(f"Wczytywanie {len(xls_files)} plików z danymi karetek...")
    all_ambulance_data = [pd.read_excel(f) for f in xls_files]
    ambulance_df = pd.concat(all_ambulance_data, ignore_index=True)

    # Czyszczenie danych
    ambulance_df['Czas wezwania'] = pd.to_datetime(ambulance_df['Czas wezwania'], errors='coerce')
    ambulance_df['Czas wyjazdu ZRM'] = pd.to_datetime(ambulance_df['Czas wyjazdu ZRM'], errors='coerce')
    ambulance_df['Czas powrotu ZRM'] = pd.to_datetime(ambulance_df['Czas powrotu ZRM'], errors='coerce')

    # Uzupełnienie brakujących czasów wyjazdu
    mask_missing_departure = ambulance_df['Czas wyjazdu ZRM'].isna()
    ambulance_df.loc[mask_missing_departure, 'Czas wyjazdu ZRM'] = ambulance_df.loc[mask_missing_departure, 'Czas wezwania']

    # Usunięcie braków
    ambulance_df = ambulance_df.dropna(subset=['Czas wyjazdu ZRM', 'Czas powrotu ZRM'])

    # Dodanie wyjazd_id
    ambulance_df = ambulance_df.reset_index(drop=True)
    ambulance_df['wyjazd_id'] = ambulance_df.index.map(lambda i: f"WYZ_{i+1:05}")

    # Ładowanie plików GPS
    gps_files = get_all_files(gps_dir, '.csv')
    if not gps_files:
        raise ValueError("Brak plików .csv z danymi GPS!")

    print(f"Wczytywanie {len(gps_files)} plików z danymi GPS...")
    gps_data = pd.concat([pd.read_csv(f) for f in gps_files], ignore_index=True)

    # Formatowanie daty i filtrowanie braków
    gps_data['GPS_TIME'] = pd.to_datetime(gps_data['GPS_TIME'], errors='coerce')
    gps_data = gps_data.dropna(subset=['GPS', 'GPS_TIME'])
    gps_data['GPS'] = gps_data['GPS'].astype(str)
    gps_data = gps_data.sort_values('GPS_TIME')

    # Grupowanie po ID_GPS
    gps_by_id = dict(tuple(gps_data.groupby('GPS')))
    #print(gps_by_id)

    print("Przetwarzanie wyjazdów i dopasowywanie danych GPS...")

    combined_data = []

    for _, row in ambulance_df.iterrows():

        id_gps = str((row['ID_GPS'])).strip().replace(".0","")
        czas_start = row['Czas wyjazdu ZRM']
        czas_koniec = row['Czas powrotu ZRM']
        wyjazd_id = row['wyjazd_id']
        #print(id_gps)
        gps_group = gps_by_id.get(id_gps)

        if gps_group is None:
            continue

        gps_group = gps_group.set_index('GPS_TIME', drop=False)
        match = gps_group.loc[czas_start:czas_koniec]


        for _, gps_row in match.iterrows():
            combined_data.append({
                'wyjazd_id': wyjazd_id,
                'ID_GPS': id_gps,
                'Czas wyjazdu ZRM': czas_start,
                'Czas powrotu ZRM': czas_koniec,
                'Powód wezwania': row.get('Powód wezwania', ''),
                'Kod pilności': row.get('Kod pilności', ''),
                'Rodzaj wyjazdu': row.get('Rodzaj wyjazdu 0- na sygnale, 1 -zwykly', ''),
                'Typ zespolu': row.get('Typ zespolu', ''),
                'Określenie wieku pacjenta': row.get('Określenie wieku pacjenta 0- dziecko, 1 - dorosly', ''),
                'GPS_TIME': gps_row['GPS_TIME'],
                'GPS_LAT': gps_row.get('GPS_LAT', ''),
                'GPS_LON': gps_row.get('GPS_LON', ''),
                'GPS_SPEED': gps_row.get('GPS_SPEED', ''),
                'SYGNALIZACJA_SWIETLNA': gps_row.get('SYGNALIZACJA_SWIETLNA', ''),
                'SYGNALIZACJA_DZWIEKOWA': gps_row.get('SYGNALIZACJA_DZWIEKOWA', ''),
            })

    # Zapisz wynik do pliku
    result_df = pd.DataFrame(combined_data)
    result_df.to_csv(output_file, index=False)
    print(f"\nZapisano {len(result_df)} rekordów do pliku: {output_file}")


In [48]:
process_data('/content/gps', '/content/kzw')

Wczytywanie 2 plików z danymi karetek...
Wczytywanie 3 plików z danymi GPS...
Przetwarzanie wyjazdów i dopasowywanie danych GPS...

Zapisano 481685 rekordów do pliku: wynik.csv
